In [42]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  

In [43]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [44]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [45]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization", "other"]

rows = []

In [46]:
norm=False

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[143.8888, 127.1705, 117.5357], std=[69.8313, 64.5137, 66.9933])])
else:
   # transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    transform=None

train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=2)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=2)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=2)

Files already downloaded and verified
Files already downloaded and verified


In [47]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)
undersampler_smote = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * 50 * NUM_CLASSES_REDUCED), replacement=False)
weight *= class_count[0]

[5000   50]


In [48]:
train_loader = DataLoader(train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_smote_undersampled = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler_smote)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [49]:
# to be used in distance capped smote - get average tensor for the entire class 
dataset = train_loader_ratio.dataset
class0 = dataset.images[dataset.labels==0]
class1 = dataset.images[dataset.labels==1]
class0_avg = torch.mean(class0.float(), 0)
class1_avg = torch.mean(class1.float(), 0)
class_img_list = [class0, class1]
avg_tensors_list = [class0_avg, class1_avg]

In [40]:
# triplet loss first few epochs + capped smote (cosine distance)

# 2 class triplet loss with capped SMOTE 


momentum=0
learning_rates = [(5e-6, 1e-2), (1e-6, 5e-3), (1e-6, 1e-2), (5e-6, 5e-2)]

cap_aucs = []
loss_fn_args = {}
loss_caps = [0.75, 1, 5]
torch.autograd.set_detect_anomaly(False)

loss_fn_args['distance'] = 'cosine'
start_epoch = 5

for cap in loss_caps:
    learning_rate_aucs = []
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            best_embed_network = None 
            best_loss = 100000000
            model_aucs = []
            embed_network = models.ConvNetOnlyEmbeddings(2)
            linear_probe = models.ConvNetLinearProbe(2)
            complete_network = models.CompleteConvNet(embed_network, linear_probe)
            embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
            linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
                print("Train loss: Avg. loss: " + str(np.mean(np.array(train_losses)) / len(train_loader.dataset)))
                test_losses = metric_utils.triplet_loss(test_loader_tripletloss, embed_network)
                if (test_losses[0] < best_loss and test_losses != 0):
                    best_embed_network = copy.deepcopy(embed_network)
                    best_loss = test_losses[0]
            for epoch in range(start_epoch, n_epochs+1):
                complete_network = models.CompleteConvNet(best_embed_network, linear_probe)
                loss_fn_args['loss_cap'] = cap
                loss_fn_args['avg_tensors'] = []
                for k in range(2):
                    avg_tensor = best_embed_network(avg_tensors_list[k])
                    loss_fn_args['avg_tensors'].append(avg_tensor)
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, complete_network, linear_optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                    model_aucs.append(auc) 
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    cap_aucs.append([auc_mean, auc_variance])



for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote_with_smote_triplet_loss", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, "start_epoch=5"]
        rows.append(row)


Test set: Avg. loss: 0.0006935921907424927, AUC: 0.49950249999999996

Train loss: 2.177517574683876
Train loss: 1.6380580819336472
Train loss: 1.1406405074581218
Train loss: 0.7840143058710037
Train loss: 0.6454909557749511
Train loss: 0.6736217581542434
Train loss: 0.5023672933791093

Test set: Avg. loss: 0.0005835582911968231, AUC: 0.82512


Test set: Avg. loss: 0.0009920628368854524, AUC: 0.8249149999999998


Test set: Avg. loss: 0.0012335069775581359, AUC: 0.825456


Test set: Avg. loss: 0.0006931573152542114, AUC: 0.500922

Train loss: 1.6729937947479783
Train loss: 1.2497364237050341
Train loss: 1.4129337026814746
Train loss: 0.8325555635865327
Train loss: 0.8456783746458163
Train loss: 0.6941262654438141
Train loss: 0.4846640954351729

Test set: Avg. loss: 0.0005466402471065522, AUC: 0.834989


Test set: Avg. loss: 0.0006964411139488221, AUC: 0.864082


Test set: Avg. loss: 0.0011362184882164, AUC: 0.8568830000000001


Test set: Avg. loss: 0.0006933403015136719, AUC: 0.49122499

Train loss: 1.2180204220638153
Train loss: 0.9437161213273455
Train loss: 1.1438231540333694
Train loss: 1.0984935316310567
Train loss: 0.9357851470351979

Test set: Avg. loss: 0.000600229024887085, AUC: 0.7645534999999999


Test set: Avg. loss: 0.0005565653443336487, AUC: 0.862451


Test set: Avg. loss: 0.0008587767779827118, AUC: 0.863323


Test set: Avg. loss: 0.0006796721220016479, AUC: 0.6995895000000001

Train loss: 0.9809628584582335
Train loss: 1.484111073670114
Train loss: 1.4938363877071696
Train loss: 1.3425806813938603
Train loss: 0.9585896707644128
Train loss: 1.0558493873875612
Train loss: 0.8446820545348392

Test set: Avg. loss: 0.000575296938419342, AUC: 0.789242


Test set: Avg. loss: 0.0006668233573436737, AUC: 0.8551500000000001


Test set: Avg. loss: 0.000939850389957428, AUC: 0.8261339999999999


Test set: Avg. loss: 0.0006948827803134918, AUC: 0.4687185

Train loss: 0.8151624411534352
Train loss: 0.629118713983305
Train loss: 0.9305645246414622
Train loss: 0.98903


Test set: Avg. loss: 0.0005970579981803894, AUC: 0.87058


Test set: Avg. loss: 0.001575674593448639, AUC: 0.5713649999999999


Test set: Avg. loss: 0.0021874072551727295, AUC: 0.850484


Test set: Avg. loss: 0.0006925201416015625, AUC: 0.6156795

Train loss: 0.9343462924289095
Train loss: 0.6148989853585601
Train loss: 0.44618014127585537
Train loss: 0.4370109970402566
Train loss: 0.2451456701679594
Train loss: 0.236677428339697
Train loss: 0.4134507080551925

Test set: Avg. loss: 0.0006584177911281585, AUC: 0.858113


Test set: Avg. loss: 0.001124662458896637, AUC: 0.8633175000000001


Test set: Avg. loss: 0.0019451584815979004, AUC: 0.85242


Test set: Avg. loss: 0.0006994863748550415, AUC: 0.31585050000000003

Train loss: 3.351314946344704
Train loss: 2.988919793040889
Train loss: 1.928568492649467
Train loss: 1.624183717047333
Train loss: 1.2383005167268644
Train loss: 0.948305770849726
Train loss: 1.1118718298377506

Test set: Avg. loss: 0.0008737375140190124, AUC: 0.80977200000

Train loss: 1.2023347259327104
Train loss: 1.1260886385941962
Train loss: 1.2624305676502787
Train loss: 1.3076492407519347
Train loss: 1.6729817743513995
Train loss: 0.932061409115032
Train loss: 1.1930139406471496

Test set: Avg. loss: 0.0005784616768360138, AUC: 0.790336


Test set: Avg. loss: 0.0005655935108661652, AUC: 0.8682270000000001


Test set: Avg. loss: 0.000827675849199295, AUC: 0.8507750000000001


Test set: Avg. loss: 0.00069707390666008, AUC: 0.5197765

Train loss: 1.2094121252655223
Train loss: 1.215760505123503
Train loss: 1.1046511136042845
Train loss: 1.806936825916266
Train loss: 1.2945188477540472
Train loss: 1.0583949100439716
Train loss: 1.2277944441054278

Test set: Avg. loss: 0.0006097620129585266, AUC: 0.774412


Test set: Avg. loss: 0.0007776130139827729, AUC: 0.834867


Test set: Avg. loss: 0.0009882590174674988, AUC: 0.8569760000000001


Test set: Avg. loss: 0.000694716602563858, AUC: 0.480065

Train loss: 1.8725317409084101
Train loss: 2.1502137772596566


Train loss: 1.2612491887845811
Train loss: 1.1313921216946499

Test set: Avg. loss: 0.0005779439210891724, AUC: 0.8200909999999999


Test set: Avg. loss: 0.0007442227005958557, AUC: 0.8520730000000001


Test set: Avg. loss: 0.0009915706515312194, AUC: 0.866611


Test set: Avg. loss: 0.0006961906254291534, AUC: 0.356847

Train loss: 1.6302833951962221
Train loss: 1.1831953927969476
Train loss: 1.3946104983615268
Train loss: 0.655754076827104
Train loss: 0.6458332158957317
Train loss: 0.5085545391034169
Train loss: 0.5506968760186699

Test set: Avg. loss: 0.0006036537587642669, AUC: 0.8844140000000001


Test set: Avg. loss: 0.001022331804037094, AUC: 0.874194


Test set: Avg. loss: 0.002027942895889282, AUC: 0.8447259999999999


Test set: Avg. loss: 0.0006906534433364868, AUC: 0.594153

Train loss: 1.3279404207399697
Train loss: 1.2319500503266694
Train loss: 0.741771079172754
Train loss: 0.9240231597499483
Train loss: 0.9467883056895748
Train loss: 0.8489766037388212
Train loss: 0.86996


Test set: Avg. loss: 0.0009700717628002166, AUC: 0.8710479999999999


Test set: Avg. loss: 0.0007009858787059784, AUC: 0.274546

Train loss: 1.0191231256078004
Train loss: 0.8665515875360769
Train loss: 0.5478816286773439
Train loss: 0.5934320802141906
Train loss: 0.4585166429258456
Train loss: 0.36518256527603055
Train loss: 0.2766122594001187

Test set: Avg. loss: 0.0005422953069210053, AUC: 0.8300485


Test set: Avg. loss: 0.0009197070300579071, AUC: 0.862573


Test set: Avg. loss: 0.0007927222549915314, AUC: 0.891192


Test set: Avg. loss: 0.0006937234699726105, AUC: 0.505826

Train loss: 1.78541615282654
Train loss: 1.1110326120048573
Train loss: 1.062967714610373
Train loss: 0.3780218108444457
Train loss: 0.6052714833028757
Train loss: 0.3628141678822268
Train loss: 0.4713394584929108

Test set: Avg. loss: 0.000610990047454834, AUC: 0.8591884999999999


Test set: Avg. loss: 0.0008457472026348114, AUC: 0.8621699999999999


Test set: Avg. loss: 0.0012760106921195983, AUC: 0.827220

Train loss: 3.73941829250117
Train loss: 3.0514345605661917
Train loss: 3.104643801974643
Train loss: 2.612534505546473
Train loss: 3.0523950939725157
Train loss: 2.736357009714576
Train loss: 2.487650365206846

Test set: Avg. loss: 0.0005795770883560181, AUC: 0.811379


Test set: Avg. loss: 0.0009680385887622833, AUC: 0.834955


Test set: Avg. loss: 0.0014414542317390442, AUC: 0.812766


Test set: Avg. loss: 0.0007007932364940644, AUC: 0.3357205

Train loss: 2.0767022090353024
Train loss: 2.7588721627642396
Train loss: 2.3932931764869934
Train loss: 1.988753900406467
Train loss: 1.9661894076189417
Train loss: 1.6306139564817879
Train loss: 1.5500993246485473

Test set: Avg. loss: 0.0005616866648197174, AUC: 0.8354760000000001


Test set: Avg. loss: 0.0007120003402233124, AUC: 0.86705


Test set: Avg. loss: 0.0011443256735801696, AUC: 0.848986


Test set: Avg. loss: 0.0006950245201587677, AUC: 0.411188

Train loss: 0.8769399899585991
Train loss: 1.06430680243073
Train loss: 0.918808770

In [41]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [52]:
# cosine distance + capped loss using whole class average tensor
momentum=0
learning_rates = [5e-4, 1e-3]


cap_aucs=[]

    
loss_fn_args = {}
loss_caps = [0.75]
loss_fn_args['distance'] = 'cosine'

start_epoch = 2

learning_rate_aucs = []

for loss_cap in loss_caps:
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(100):
            model_aucs = []
            network = models.ConvNetWithEmbeddingsEarly(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                loss_fn_args['avg_tensors'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False,loss_fn=loss_fns.CappedBCELossAvgDistance,loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                loss_fn_args['print_loss']= False
                loss_fn_args['avg_tensors'] = []
                for k in range(NUM_CLASSES_REDUCED):
                    _, avg_tensor = network(avg_tensors_list[k])
                    loss_fn_args['avg_tensors'].append(avg_tensor)
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])

for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote_avg", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, 'num_models=100']
        rows.append(row)


Test set: Avg. loss: 0.002039885640144348, AUC: 0.66023


Test set: Avg. loss: 0.0006628685593605041, AUC: 0.7356530000000002


Test set: Avg. loss: 0.0006643344759941102, AUC: 0.773584


Test set: Avg. loss: 0.0006539630889892578, AUC: 0.8037645


Test set: Avg. loss: 0.0047306792736053465, AUC: 0.6329575000000001


Test set: Avg. loss: 0.0005529289841651917, AUC: 0.878345


Test set: Avg. loss: 0.0010869187712669373, AUC: 0.850248


Test set: Avg. loss: 0.0008945511877536773, AUC: 0.8861870000000001


Test set: Avg. loss: 0.001632814586162567, AUC: 0.6211545


Test set: Avg. loss: 0.0006615898311138153, AUC: 0.7973415000000001


Test set: Avg. loss: 0.0006592643857002258, AUC: 0.8105370000000001


Test set: Avg. loss: 0.0006474920809268951, AUC: 0.8341635000000001


Test set: Avg. loss: 0.0011176564693450928, AUC: 0.5269135


Test set: Avg. loss: 0.0006419839262962341, AUC: 0.8088095


Test set: Avg. loss: 0.0006355904936790466, AUC: 0.8086760000000001


Test set: Avg. loss: 0.00068


Test set: Avg. loss: 0.003161272644996643, AUC: 0.41251450000000006


Test set: Avg. loss: 0.0006577843129634858, AUC: 0.7721280000000001


Test set: Avg. loss: 0.0006510413885116577, AUC: 0.8119590000000001


Test set: Avg. loss: 0.0006834783256053924, AUC: 0.7371965


Test set: Avg. loss: 0.0007635658979415894, AUC: 0.5259750000000001


Test set: Avg. loss: 0.0006667162775993347, AUC: 0.826497


Test set: Avg. loss: 0.00046142929792404174, AUC: 0.8826079999999998


Test set: Avg. loss: 0.0005662816166877747, AUC: 0.8821570000000001


Test set: Avg. loss: 0.00527819013595581, AUC: 0.623162


Test set: Avg. loss: 0.000671224981546402, AUC: 0.7465020000000001


Test set: Avg. loss: 0.0006038239300251007, AUC: 0.818011


Test set: Avg. loss: 0.0006939477026462555, AUC: 0.763509


Test set: Avg. loss: 0.0009949098825454711, AUC: 0.525459


Test set: Avg. loss: 0.0005505824089050293, AUC: 0.8350599999999999


Test set: Avg. loss: 0.0006728788614273071, AUC: 0.851826


Test set: Avg. loss:


Test set: Avg. loss: 0.0006707564294338226, AUC: 0.8111705


Test set: Avg. loss: 0.009831798553466798, AUC: 0.4117655


Test set: Avg. loss: 0.0005761415660381317, AUC: 0.855991


Test set: Avg. loss: 0.0006893740892410278, AUC: 0.8189139999999999


Test set: Avg. loss: 0.0005601320862770081, AUC: 0.8692980000000001


Test set: Avg. loss: 0.00261846125125885, AUC: 0.5577


Test set: Avg. loss: 0.0006089528501033783, AUC: 0.838268


Test set: Avg. loss: 0.0006422235369682312, AUC: 0.825171


Test set: Avg. loss: 0.0011870046257972718, AUC: 0.8056099999999999


Test set: Avg. loss: 0.0015995723605155944, AUC: 0.4268235


Test set: Avg. loss: 0.0005534631311893463, AUC: 0.865054


Test set: Avg. loss: 0.0005400881469249725, AUC: 0.8696229999999999


Test set: Avg. loss: 0.0006624490320682525, AUC: 0.835174


Test set: Avg. loss: 0.0015393699407577515, AUC: 0.4664165


Test set: Avg. loss: 0.0006816013753414154, AUC: 0.5793659999999999


Test set: Avg. loss: 0.0006377974450588226, AUC: 0


Test set: Avg. loss: 0.0006500383615493775, AUC: 0.7805845


Test set: Avg. loss: 0.000675906240940094, AUC: 0.7388090000000002


Test set: Avg. loss: 0.0011087210178375244, AUC: 0.6278115


Test set: Avg. loss: 0.0005141288638114929, AUC: 0.8803750000000001


Test set: Avg. loss: 0.0006745084524154663, AUC: 0.8822060000000002


Test set: Avg. loss: 0.0009938264191150666, AUC: 0.873124


Test set: Avg. loss: 0.0016988049745559693, AUC: 0.5635330000000001


Test set: Avg. loss: 0.0006558447182178497, AUC: 0.7788655000000001


Test set: Avg. loss: 0.000636860877275467, AUC: 0.839542


Test set: Avg. loss: 0.000644484668970108, AUC: 0.873401


Test set: Avg. loss: 0.0029737695455551148, AUC: 0.4846755


Test set: Avg. loss: 0.0004959933757781983, AUC: 0.8758


Test set: Avg. loss: 0.0009878093600273132, AUC: 0.8573959999999999


Test set: Avg. loss: 0.00044674426317214966, AUC: 0.8777995


Test set: Avg. loss: 0.0014749653935432433, AUC: 0.38021


Test set: Avg. loss: 0.00064219722151756


Test set: Avg. loss: 0.00048768319189548495, AUC: 0.878092


Test set: Avg. loss: 0.00044023710489273074, AUC: 0.892667


Test set: Avg. loss: 0.0016253237724304199, AUC: 0.531132


Test set: Avg. loss: 0.0006592078804969788, AUC: 0.8558719999999999


Test set: Avg. loss: 0.0006208440065383911, AUC: 0.870744


Test set: Avg. loss: 0.0006931117773056031, AUC: 0.8311540000000001


Test set: Avg. loss: 0.0056273510456085205, AUC: 0.3395535


Test set: Avg. loss: 0.0006935471892356872, AUC: 0.49993350000000003


Test set: Avg. loss: 0.000693195253610611, AUC: 0.500504


Test set: Avg. loss: 0.0009124216139316559, AUC: 0.820488


Test set: Avg. loss: 0.001134386420249939, AUC: 0.5104730000000001


Test set: Avg. loss: 0.000614866316318512, AUC: 0.8556785000000001


Test set: Avg. loss: 0.0006206272244453431, AUC: 0.856273


Test set: Avg. loss: 0.000652146726846695, AUC: 0.858349


Test set: Avg. loss: 0.0009268079102039337, AUC: 0.508106


Test set: Avg. loss: 0.0005959088802337646, AUC: 


Test set: Avg. loss: 0.0006895624101161957, AUC: 0.8010490000000001


Test set: Avg. loss: 0.0007369867265224456, AUC: 0.7629954999999999


Test set: Avg. loss: 0.0012457998394966125, AUC: 0.555586


Test set: Avg. loss: 0.0006952352225780488, AUC: 0.65932


Test set: Avg. loss: 0.0006939291656017304, AUC: 0.7795179999999999


Test set: Avg. loss: 0.0007086337804794312, AUC: 0.794881


Test set: Avg. loss: 0.0019257506132125855, AUC: 0.454948


Test set: Avg. loss: 0.0006175247728824615, AUC: 0.8016159999999999


Test set: Avg. loss: 0.0006350263059139252, AUC: 0.799287


Test set: Avg. loss: 0.0006606808602809906, AUC: 0.791172


Test set: Avg. loss: 0.0008123100697994232, AUC: 0.6383375


Test set: Avg. loss: 0.0006826936006546021, AUC: 0.708519


Test set: Avg. loss: 0.000645332008600235, AUC: 0.8176844999999999


Test set: Avg. loss: 0.0006571354866027832, AUC: 0.81378


Test set: Avg. loss: 0.001607116162776947, AUC: 0.379599


Test set: Avg. loss: 0.0006350829601287842, AUC: 0.8


Test set: Avg. loss: 0.0006394362151622773, AUC: 0.8318735


Test set: Avg. loss: 0.0006617222726345062, AUC: 0.8666920000000001


Test set: Avg. loss: 0.0007403411865234375, AUC: 0.8690680000000001


Test set: Avg. loss: 0.0013845330476760865, AUC: 0.5036105


Test set: Avg. loss: 0.0006082592308521271, AUC: 0.845541


Test set: Avg. loss: 0.0005472329258918762, AUC: 0.865144


Test set: Avg. loss: 0.0006499690711498261, AUC: 0.8656619999999999


Test set: Avg. loss: 0.0010931934118270873, AUC: 0.329523


Test set: Avg. loss: 0.0006692779362201691, AUC: 0.840603


Test set: Avg. loss: 0.0008829580843448639, AUC: 0.8413059999999999


Test set: Avg. loss: 0.0006391588747501374, AUC: 0.802193


Test set: Avg. loss: 0.002035938858985901, AUC: 0.336951


Test set: Avg. loss: 0.0006522195637226105, AUC: 0.872225


Test set: Avg. loss: 0.001666193962097168, AUC: 0.825898


Test set: Avg. loss: 0.0009747958779335022, AUC: 0.8789880000000001


Test set: Avg. loss: 0.0009209502041339874, AUC: 

In [53]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [ ]:
# triplet loss smote + no smote training 


momentum=0
learning_rates = [(5e-6, 5e-3), (1e-6, 1e-2), (5e-6, 1e-3), (1e-6, 5e-4)]

loss_fn_args = {}
torch.autograd.set_detect_anomaly(False)


start_epoch = 10


learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddingsEarly(2)
        linear_probe = models.ConvNetLinearProbeEarly(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        for epoch in range(start_epoch, n_epochs+1):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, complete_network, linear_optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)




for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, "start_epoch=10"]
    rows.append(row)

In [ ]:
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

In [ ]:
for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], cap, norm, "start_epoch=10"]
    rows.append(row)

In [32]:

for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, "start_epoch=10"]
    rows.append(row)

In [33]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [36]:
# 2 class triplet loss no ratio 
# no smote 


momentum=0
learning_rates = [(5e-6, 1e-2), (1e-6, 5e-3), (1e-6, 1e-2), (5e-6, 5e-2)]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddingsEarly(2)
        linear_probe = models.ConvNetLinearProbeEarly(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)
    
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)
    
    



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0006957003772258758, AUC: 0.39529899999999996

Train loss: 2.310011562648093
Train loss: 2.2350221542036457
Train loss: 1.7336593600595074
Train loss: 1.3570892761467368
Train loss: 1.7184112235239357
Train loss: 1.1358157290015252
Train loss: 1.0679599012538885
Train loss: 0.921282701431566
Train loss: 1.1848033970328653
Train loss: 0.9810460426245525
Train loss: 0.9329271555706194
Train loss: 0.9048631927769655
Train loss: 0.5647244722979843
Train loss: 0.5945013852635767
Train loss: 0.5329235454273832

Test set: Avg. loss: 0.00039782971143722536, AUC: 0.918406


Test set: Avg. loss: 0.0004035674780607224, AUC: 0.927185


Test set: Avg. loss: 0.00037781354784965514, AUC: 0.935988


Test set: Avg. loss: 0.0006927546858787537, AUC: 0.530769

Train loss: 2.022161154610336
Train loss: 1.288486762031628
Train loss: 1.399418694957806
Train loss: 1.0709760223224665
Train loss: 0.8879150535650314
Train loss: 0.994375331386639
Train loss: 0.9763853713205666
Train loss:

Train loss: 1.970422636551462
Train loss: 1.4079155132269403
Train loss: 1.4688413837912735
Train loss: 1.4076758281440491
Train loss: 1.1350592856953858
Train loss: 1.8857001426872935
Train loss: 1.4556400149491182

Test set: Avg. loss: 0.00047363899648189545, AUC: 0.8819830000000001


Test set: Avg. loss: 0.00042409318685531617, AUC: 0.908577


Test set: Avg. loss: 0.0003766205757856369, AUC: 0.922854


Test set: Avg. loss: 0.0006900011897087098, AUC: 0.654073

Train loss: 1.9343711790765168
Train loss: 1.4965036446881141
Train loss: 1.744480819459174
Train loss: 1.445263187217105
Train loss: 1.3542424216391935
Train loss: 1.3084997363910553
Train loss: 1.4788874330793975
Train loss: 1.3785485469611587
Train loss: 1.6266570197548835
Train loss: 1.6530657175240244
Train loss: 1.117041161105891
Train loss: 1.1217445317347339
Train loss: 0.7305867747896037
Train loss: 1.335534641317501
Train loss: 1.165849501539947

Test set: Avg. loss: 0.0004729681611061096, AUC: 0.886551


Test set: A


Test set: Avg. loss: 0.00036962753534317016, AUC: 0.9312050000000001


Test set: Avg. loss: 0.0004138304591178894, AUC: 0.9353440000000002


Test set: Avg. loss: 0.000701473593711853, AUC: 0.40515

Train loss: 1.065467907744608
Train loss: 1.3622498276886668
Train loss: 1.5146699308589766
Train loss: 1.1519298059925152
Train loss: 1.2872650144965785
Train loss: 1.1337903233090783
Train loss: 1.0842262722884015
Train loss: 1.1602510191073083
Train loss: 0.7593976547763606
Train loss: 1.2732060164403005
Train loss: 1.3662886513266594
Train loss: 0.6023548021438015
Train loss: 1.124361228791012
Train loss: 1.1234989906572233
Train loss: 0.8597518346112245

Test set: Avg. loss: 0.00041623535752296446, AUC: 0.920745


Test set: Avg. loss: 0.0003963103443384171, AUC: 0.934286


Test set: Avg. loss: 0.00045862875878810883, AUC: 0.936785


Test set: Avg. loss: 0.0006941128373146058, AUC: 0.5132665

Train loss: 3.3644635339451443
Train loss: 2.1163280215232994
Train loss: 1.979751812044982
Tra

Train loss: 1.9346731878389978
Train loss: 1.5430769199019025
Train loss: 1.4607565668737812
Train loss: 1.393643997277424
Train loss: 1.3211874517665547
Train loss: 0.921329250381251
Train loss: 0.9669004671133248
Train loss: 0.9917047209800429
Train loss: 0.7282513793866345
Train loss: 1.0467327958459307
Train loss: 0.5617396865680719
Train loss: 0.876054488929214
Train loss: 0.6982500355714446

Test set: Avg. loss: 0.00039860232174396515, AUC: 0.9290079999999998


Test set: Avg. loss: 0.00047183363139629365, AUC: 0.937491


Test set: Avg. loss: 0.00032021838426589963, AUC: 0.948357


Test set: Avg. loss: 0.0006923727095127105, AUC: 0.5628065

Train loss: 1.5512303265796346
Train loss: 1.2353783249855042
Train loss: 1.1475682938174836
Train loss: 1.4969956195278533
Train loss: 0.979395692515525
Train loss: 1.053526867726806
Train loss: 0.7830445409580401
Train loss: 0.7794080646174728
Train loss: 0.9118682161258285
Train loss: 0.7145632490230973
Train loss: 0.9021045573198112
Train l

In [37]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []